<a href="https://colab.research.google.com/github/xper100/Project_Review/blob/main/Project(Wiseapp).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> 문제정의: 분류가 되어있지 않음, 

> 현재상황: 정규식으로 분류하여 정확도가 90%임, 정확도가 생명이기에 정확도가 높을 수록 좋음

> 결재금액, 카드메시지, 브랜드명 => 첫줄 15000, (주)우아한형제, 미분류

> 매출액 상위 1000? 정도만 관리하고자 함. 점유율이 낮은 브랜드는 제외해도 됨

> 매출액에 따라 분류작업 -> 분류작업하려는 브랜드의 총금액 비율에서: 건수 50%, 금액 30%

> 프로젝트 요약

* 이미 분류가 되어 있는 브랜드 => 정확성을 높이고,
* 미분류가 되어 있는 브랜드 => 분류할 수 있도록

---

**주관심**: 기술의 성능 인사이트, 서비스에 사용가능 여부

Minimum: 현재 수작업을 줄여줄 수 있는 기술 필요 -> 클러스터링을 통해 작업 수월하게

Best: 정확성이 높은 분류기술

---

**전처리**: 수작업으로 작업된 Rule을 통해 분류 가능

* 미리 분류된 데이터를 통해 타겟을 줄인 후 진행하는 편이 수월함

* 이전에 Supervised Learning을 써야할 것 같음

* 자연어처리

* 가능성을 보기 위한 프로젝트로서 작업 시작






In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# # path = '/content/drive/MyDrive/Colab Notebooks/AI부트캠프/기업협업 프로젝트 (아이디어웨어)/data/'

# 데이터 불러오기

In [33]:
import pandas as pd
import numpy as np
import re

# 경로 설정 및 컬럼명 수정
path = '/Users/xper100/codestates/project2/'
col_name = ['체크일자', '데이터타입', '패널', '메시지', '업체 메시지', '결제방법', '사용일시', '카드승인타입',
            '카드사명', '결제금액', '결제금액2', '카드메시지','카드메시지2', '사용가능 여부','브랜드',
            '브랜드명','할부개월수','업체 분류 브랜드','주소','위도', '경도', '간편결제여부','페이먼트 종류',
            '페이먼트 종류_매칭용', '간편결제 가맹점명', '간편결제 사용내역', '간편결제 문자 아이디',
            '체크카드계좌 문자 아이디', '체크카드계좌 가맹점명','문자타입','서비스타입','등록일자']
# 데이터 불러오기            
df = pd.read_csv(path + '2021-07-01.csv', names = col_name, index_col = False)
df
# use_col = ['결제금액2','카드메시지', '브랜드명']
# df_use = df.loc[:,use_col]
# df_use.head()

/Users/xper100/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (14,16,21,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/xper100/opt/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


,체크일자,데이터타입,패널,메시지,업체 메시지,결제방법,사용일시,카드승인타입,카드사명,결제금액,결제금액2,카드메시지,카드메시지2,사용가능 여부,브랜드,브랜드명,할부개월수,업체 분류 브랜드,주소,위도,경도,간편결제여부,페이먼트 종류,페이먼트 종류_매칭용,간편결제 가맹점명,간편결제 사용내역,간편결제 문자 아이디,체크카드계좌 문자 아이디,체크카드계좌 가맹점명,문자타입,서비스타입,등록일자
0,2021-07-01,1,50157dcd2ef9e222029a8ae2daacc50d,00000a07e0a5056ada7fd9a5ff06de8f,\N,1,2021-07-01 21:33:11,1,우리,15000,15000,(주)우아한형제들,(주)우아한형제들,\N,16985,배달의민족,1,\N,\N,\N,\N,0,\N,\N,\N,\N,\N,\N,\N,\N,\N,1
1,2021-07-01,1,42b89bf7f9c5988d87d192b8215ec82b,0000119de131183f4e4c4de9c1f281fa,\N,5,2021-07-01 16:10:11,1,NH농협,15000,15000,이미숙6월캡,이미숙6월캡,\N,-1,미분류,1,\N,\N,\N,\N,0,\N,\N,\N,\N,\N,\N,\N,\N,\N,1
2,2021-07-01,1,8dc66a909acf2b4ccda1c4c0a53fb382,000015e2dcefad349b40457a86d5fdcb,\N,1,2021-07-01 22:15:03,1,신한,3000,3000,GS25중곡덕산,GS25중곡덕산,\N,10354,GS25,1,\N,\N,\N,\N,0,\N,\N,\N,\N,\N,\N,\N,\N,\N,2
3,2021-07-01,1,2ac7cf1f00570f2ecc0da8e7ea49b740,00003960607638c986f379be322c29c7,\N,5,2021-07-01 15:17:43,1,NH농협,200000,200000,경주페이,경주페이,\N,-1,미분류,1,\N,\N,\N,\N,0,\N,\N,\N,\N,\N,\N,\N,\N,\N,2
4,2021-07-01,1,d3d7778911ebd2007519490fe4cc17c2,00003ebd11936347acfa98e28f57197e,\N,1,2021-07-01 13:46:53,1,우리,32880,32880,하이웨이마트송탄지*,하이웨이마트송탄지*,\N,18710,하이웨이마트,1,\N,\N,\N,\N,0,\N,\N,\N,\N,\N,\N,\N,\N,\N,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2421100,2021-07-01,1,0b13c553a8342384b7fd91dfcbbfe593,ffffe099b42d5b545523b1b450046400,\N,1,2021-07-01 15:03:41,1,삼성,3500,3500,상주농협하나로,상주농협하나로,\N,16398,농협하나로마트,1,\N,\N,\N,\N,0,\N,\N,\N,\N,\N,\N,\N,\N,\N,1
2421101,2021-07-01,1,9449e8a133db8c44e6058123d1686253,ffffe3c90dfca4c90b33cfbd3ef67b65,\N,1,2021-07-01 13:50:19,1,신한,54000,54000,이니시스-정,이니시스-정,\N,-1,미분류,1,\N,\N,\N,\N,0,\N,\N,\N,\N,\N,\N,\N,\N,\N,1
2421102,2021-07-01,1,140164be613c19f1c49caae595333c2e,ffffe6538e7d492c37429c6d0f34324e,\N,5,2021-07-01 16:29:59,1,NH농협,40000,40000,잉크벨양구점,잉크벨양구점,\N,-1,미분류,1,\N,\N,\N,\N,0,\N,\N,\N,\N,\N,\N,\N,\N,\N,2
2421103,2021-07-01,1,049a1c50ad4d13fe42a0493e78647940,ffffe77245069b7db434340c7f7e1dd5,\N,5,2021-07-01 19:13:16,1,신한,1000000,4000000,MAENWILAI,MAENWILAI,\N,-1,미분류,1,\N,\N,\N,\N,0,\N,\N,\N,\N,\N,\N,\N,\N,\N,1


# EDA

In [55]:
cond = (df['브랜드명'] == '미분류')
df_test = (df[cond].groupby('카드메시지')
            .agg({'카드메시지2':'count', '결제금액2': 'sum'})
            .reset_index()
            .rename(columns = {'카드메시지2':'count', '결제금액2': 'sum'})
          )
cond2 = (df_test['count'] >= 20)
df_test[cond2].sort_values(by='sum', ascending = True)


,카드메시지,count,sum
255666,벤도피아,22,12400
99975,경남유통,22,12550
449947,제일테크,24,13700
355667,에이엔디코리아,22,15450
353994,에스에스벤딩,38,16200
...,...,...,...
259524,본사,67,6702185021
329137,신한카드,8133,7114610075
277146,삼성카드,8415,7834525819
458347,주식,30,14598085244


In [56]:
cond_count1 = (df_test['count'] < 2)
print("카드메시지 빈도 수가 1번인 데이터의 양: ", len(df_test[cond_count1]))
print("카드메시지 빈도 수가 1번인 데이터의 비율: {0}%".format(round( 100 * len(df_test[cond_count1])/ len(df_test), 2)) )
print("")

카드메시지 빈도 수가 1번인 데이터의 양:  442301
카드메시지 빈도 수가 1번인 데이터의 비율: 75.39%



In [57]:
len(df_test)

586673

In [58]:
df_sample = pd.read_csv(path + '2021-07-01-sample.csv', names = col_name, index_col = False)
df_sample = df_use.loc[:,use_col]
df_sample.head()

/Users/xper100/opt/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


,결제금액2,카드메시지,브랜드명
0,15000,(주)우아한형제들,배달의민족
1,15000,이미숙6월캡,미분류
2,3000,GS25중곡덕산,GS25
3,200000,경주페이,미분류
4,32880,하이웨이마트송탄지*,하이웨이마트


In [59]:
cond = (df_use['브랜드명'] != '미분류')
df_use.loc[cond, '브랜드명'].value_counts(normalize = True) * 100

쿠팡/쿠팡이츠        7.288113
GS25           5.104109
씨유(CU)         4.962416
네이버/네이버페이      4.960106
배달의민족          3.316263
                 ...   
경동렌트카          0.000086
파로마가구(제외예정)    0.000086
에듀셀파           0.000086
우가양평해장국        0.000086
푸드플라이          0.000086
Name: 브랜드명, Length: 2499, dtype: float64

In [60]:
cond = (df_use['브랜드명'] != '미분류')
df_use.loc[cond].groupby(by = '브랜드명').sum().sort_values(by = '결제금액2', ascending = False)[:20]

,결제금액2
브랜드명,
단기카드대출(현금서비스),25593214954
"예금,적금(제외)",14604944736
MG손해보험,9461972682
롯데카드,7711336786
네이버/네이버페이,3505364911
쿠팡/쿠팡이츠,3018471540
카카오페이(카카오 통합),2777026815
카카오-제거용,2388788905
롯데(분배처리후 기타모음),2341084267


* 카드메시지를 한단어씩 분류 -> LSTM으로 ?

* 문서 군집화와 문서간 유사도 측정하기

https://techblog-history-younghunjo1.tistory.com/114?category=924148

* 자모단위 한국어 FastText <- 가장유력

https://vhrehfdl.tistory.com/77

---

> FastText

word2vec과는 다르게 FastText는 단어를 여러개로 쪼개어 세부단어(subword)를 통해 학습을 진행한다. 

* 강점 

1. OOV(Out Of Vocabulary) 대응

2. 빈도수가 적은 단어(Rare Word) 대응

* 단점

1. Hyperparameter Tuning에 민감

#### 전처리

* 카드메시지 중 숫자 또는 빈공간만 있는 데이터 제거

* 특수문자 및 특수문자 안에 있는 글씨 제거

* 단어 음절을 2,3 단위로 분리


# 전처리


In [61]:
# 특수부호 제거
df_use['카드메시지'] = df_use['카드메시지'].apply(lambda x: re.sub(r"[^ㄱ-ㅎ가-힣a-zA-Z0-9]","",x))
df_use.head(10)

,결제금액2,카드메시지,브랜드명
0,15000,주우아한형제들,배달의민족
1,15000,이미숙6월캡,미분류
2,3000,GS25중곡덕산,GS25
3,200000,경주페이,미분류
4,32880,하이웨이마트송탄지,하이웨이마트
5,12000,월드약국,약국(종합)
6,49000,주우아한형제들,배달의민족
7,16400,후불하이패스,후불하이패스
8,271750,한전정은영,미분류
9,105000,대포항회수산,미분류


## 카드메시지 중 숫자 또는 빈공간만 있는 데이터 제거

In [62]:
only_digits = [ind for ind, data in enumerate(df_use['카드메시지']) if data.isdecimal() == True]
print("숫자로만 기록된 카드메시지의 양: ",len(only_digits))

숫자로만 기록된 카드메시지의 양:  25291


In [63]:
df_use.loc[only_digits]

,결제금액2,카드메시지,브랜드명
17,15000,137006849767,미분류
78,318429,756009000018,미분류
141,20000,0665413,미분류
261,238560,6411105012,미분류
270,200000,5271181923,미분류
...,...,...,...
2420706,50000,803907112,미분류
2420794,63863,288016044391,미분류
2420837,500000,0427466,미분류
2420986,1000,48891011202821,미분류


In [64]:
df_use.drop(only_digits, axis = 0, inplace = True)
df_use.reset_index(inplace=True, drop = True)
df_use.head()

,결제금액2,카드메시지,브랜드명
0,15000,주우아한형제들,배달의민족
1,15000,이미숙6월캡,미분류
2,3000,GS25중곡덕산,GS25
3,200000,경주페이,미분류
4,32880,하이웨이마트송탄지,하이웨이마트


## 카드메시지 == 내용없음 제거

In [65]:
no_message = [ind for ind, val in enumerate(df_use['카드메시지']) if val == '내용없음']
# 제거
df_use.drop(no_message, axis = 0, inplace = True)
df_use.reset_index(inplace=True, drop = True)
df_use.head()

,결제금액2,카드메시지,브랜드명
0,15000,주우아한형제들,배달의민족
1,15000,이미숙6월캡,미분류
2,3000,GS25중곡덕산,GS25
3,200000,경주페이,미분류
4,32880,하이웨이마트송탄지,하이웨이마트


## 결제 빈도수 3이하인 카드메세지 제거

In [ ]:
cond_uncat = (df_use['브랜드명'] == '미분류')
df_use_test = (df_use.groupby('카드메시지')
               .agg({'브랜드명':'count', '결제금액2':'sum'})
               .reset_index()
               .rename(columns={'브랜드명':'count', '결제금액2':'sum'})
               )
df_use_test

,카드메시지,count,sum
0,.ㅎ,1,85
1,11회3801853,1,500000
2,11회4284783,1,150000
3,12회7584143,1,400000
4,175회486353,1,40000
...,...,...,...
843502,ｎｉｃｅ신용평가,1,539000
843503,ｓｄ６월식대아,1,361402
843504,ｓｄ６월식대（,3,2309514
843505,ｗ메디텍,1,251900


In [ ]:
df_use_test.sort_values(by = ['count'], ascending = True)

,카드메시지,count,sum
386980,신한백명숙,1,150000
466933,올라아보,1,13000
466934,올라안과,1,8300
466936,올라운더랩,1,15200
466940,올라회덮밥,1,12000
...,...,...,...
604746,주우아한형제들,14372,346917525
667040,카카오페이,23065,2768266767
160775,네이버페이,24061,1375923107
682837,쿠팡,44198,1682731221


In [ ]:
# 카드메시지가 3번만 있는 경우 모두 추출 후 제거
n = 4
cond_more = (df_use_test['count'] >= n)
list_more = df_use_test.loc[cond_more, '카드메시지'].to_list()
sum_count_more = df_use_test.loc[cond_more, 'count'].sum()
sum_amount_more = df_use_test.loc[cond_more, 'sum'].sum()

# 비율확인
print("------ 빈도수가 {0}이상인 카드메시지 정보 ------".format(n))
print("종류 갯수 {0}".format( len(list_more) ) )
print("종류 비율 {0}%".format( 100 * round(len(list_more)/df_use['카드메시지'].nunique(), 4)) )
print("데이터 비율 {0}%".format( 100 * round( sum_count_more / len(df_use), 4)) )
print("결제액 비율 {0}%".format( 100 * round( sum_amount_more / df_use['결제금액2'].sum(), 4)) )

------ 빈도수가 4이상인 카드메시지 정보 ------
종류 갯수 69267
종류 비율 8.95%
데이터 비율 60.81999999999999%
결제액 비율 30.95%


In [ ]:
# 인덱스 추출 및 데이터에서 제거
ind_onlyone = [ind for ind, val in enumerate(df_use['카드메시지']) if val not in list_more]
df_use.drop(ind_onlyone, axis = 0, inplace = True)
df_use.reset_index(drop = True, inplace = True)
df_use.head()

# 미분류 클러스터링

In [ ]:
cond_uncat = (df_use['브랜드명'] == '미분류')
df_uncat = df_use[cond_uncat].reset_index().drop('index', axis = 1)
df_uncat.head(20)

,결제금액2,카드메시지,브랜드명
0,200000,경주페이,미분류
1,22100,신한카드,미분류
2,1500,분당서울대학,미분류
3,200000,전지현,미분류
4,25000,김설아,미분류
5,24800,KB카드출금,미분류
6,22100,금성커피,미분류
7,39590,지에스칼텍,미분류
8,43587,신도시충전소,미분류
9,900,06월입출통,미분류


* 카드메세지 = 받는 업체 또는 사람? => 포스기에 등록된 아이디 / 사업자 등록 시 사업자명에 따라 달라짐?

* 결제 시 메시지 오류 가능성


https://joyhong.tistory.com/135?category=915140


## 결제 빈도수 3이하인 카드메세지 제거

In [ ]:
df_use_test = (df_uncat.groupby('카드메시지')
               .agg({'브랜드명':'count', '결제금액2':'sum'})
               .reset_index()
               .rename(columns={'브랜드명':'count', '결제금액2':'sum'})
               )
df_use_test

,카드메시지,count,sum
0,,163,55190772
1,0001씨유광주지,3,7750
2,0017카드,3,1060000
3,0021카드,3,1140000
4,0023카드,4,1570600
...,...,...,...
68943,힘찬기합소리,3,390500
68944,힘찬낙지,3,88000
68945,힘찬마트,28,315370
68946,힘찬장어,11,526000


In [ ]:
df_use_test.sort_values(by = ['count'], ascending = True)

,카드메시지,count,sum
13096,대구은행소비조,1,2000
34459,역전할머니맥,1,30900
55572,주한일테크닉스,1,33000
55643,주현대석유,1,1000000
37176,우리은행주유노,1,98000
...,...,...,...
1125,KB카드출금,4460,4445321883
64601,하나카드,6499,4225582599
30844,신한카드,8133,7114610075
24868,삼성카드,8415,7834525819


In [ ]:
# 카드메시지가 1번만 있는 경우 모두 추출 후 제거
n = 3
cond_more = (df_use_test['count'] >= n)
list_more = df_use_test.loc[cond_more, '카드메시지'].to_list()
sum_count_more = df_use_test.loc[cond_more, 'count'].sum()
sum_amount_more = df_use_test.loc[cond_more, 'sum'].sum()

# 비율확인
print("------ 빈도수가 {0}이상인 카드메시지 정보 ------".format(n))
print("종류 갯수 {0}".format( len(list_more) ) )
print("종류 비율 {0}%".format( 100 * round(len(list_more)/df_uncat['카드메시지'].nunique(), 4)) )
print("데이터 비율 {0}%".format( 100 * round( sum_count_more / len(df_uncat), 4)) )
print("결제액 비율 {0}%".format( 100 * round( sum_amount_more / df_uncat['결제금액2'].sum(), 4)) )

------ 빈도수가 3이상인 카드메시지 정보 ------
종류 갯수 68739
종류 비율 99.7%
데이터 비율 99.96000000000001%
결제액 비율 99.99%


In [ ]:
# 인덱스 추출 및 데이터에서 제거
ind_onlyone = [ind for ind, val in enumerate(df_uncat['카드메시지']) if val not in list_more]
df_uncat.drop(ind_onlyone, axis = 0, inplace = True)
df_uncat.reset_index(drop = True, inplace = True)

In [ ]:
len(df_uncat)

649246

In [ ]:
# 단어 이어 붙여서 말뭉치 만들기
df_uncat['말뭉치3'] = df_uncat['카드메시지'].apply(lambda x: 3 * (x + ','))
df_uncat['토큰화'] = df_uncat['말뭉치3'].apply(lambda x: x.split(',')) # comma로 나누기
df_uncat['토큰화'] = df_uncat['토큰화'].apply(lambda x: x[:-1]) # 마지막 comma 지우기
df_uncat['토큰화']

0               [경주페이, 경주페이, 경주페이]
1               [신한카드, 신한카드, 신한카드]
2         [분당서울대학, 분당서울대학, 분당서울대학]
3                  [전지현, 전지현, 전지현]
4                  [김설아, 김설아, 김설아]
                    ...           
649241             [문정희, 문정희, 문정희]
649242    [하나카드금융, 하나카드금융, 하나카드금융]
649243       [이니시스정, 이니시스정, 이니시스정]
649244    [잉크벨양구점, 잉크벨양구점, 잉크벨양구점]
649245          [주조이젠, 주조이젠, 주조이젠]
Name: 토큰화, Length: 649246, dtype: object

# 전처리된 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

df_uncat = pd.read_csv('미분류_전처리후.csv')
df_uncat.drop(['Unnamed: 0'], axis = 1, inplace = True)
df_uncat.head(10)

,결제금액2,카드메시지,브랜드명,말뭉치3,토큰화
0,200000,경주페이,미분류,"경주페이,경주페이,경주페이,","['경주페이', '경주페이', '경주페이']"
1,22100,신한카드,미분류,"신한카드,신한카드,신한카드,","['신한카드', '신한카드', '신한카드']"
2,1500,분당서울대학,미분류,"분당서울대학,분당서울대학,분당서울대학,","['분당서울대학', '분당서울대학', '분당서울대학']"
3,200000,전지현,미분류,"전지현,전지현,전지현,","['전지현', '전지현', '전지현']"
4,25000,김설아,미분류,"김설아,김설아,김설아,","['김설아', '김설아', '김설아']"
5,24800,KB카드출금,미분류,"KB카드출금,KB카드출금,KB카드출금,","['KB카드출금', 'KB카드출금', 'KB카드출금']"
6,22100,금성커피,미분류,"금성커피,금성커피,금성커피,","['금성커피', '금성커피', '금성커피']"
7,39590,지에스칼텍,미분류,"지에스칼텍,지에스칼텍,지에스칼텍,","['지에스칼텍', '지에스칼텍', '지에스칼텍']"
8,43587,신도시충전소,미분류,"신도시충전소,신도시충전소,신도시충전소,","['신도시충전소', '신도시충전소', '신도시충전소']"
9,900,06월입출통,미분류,"06월입출통,06월입출통,06월입출통,","['06월입출통', '06월입출통', '06월입출통']"


In [ ]:
# 빈도수 기준을 통한 데이터 필터링
def df_freq_basis(df, n): 
  # ---------------------------
  # df = dataframe name, n = the number of frequency of 카드메시지 as a threshold
  # n 이상의 빈도수를 가진 카드메시지만 추출하기
  # --------------------------- 
  # 그룹화 한 데이터프레임 저장
  df_uncat_test = (df.groupby('카드메시지')
                .agg({'브랜드명':'count', '결제금액2':'sum'})
                .reset_index()
                .rename(columns={'브랜드명':'count', '결제금액2':'sum'})
                )
  # 빈도수에 따른 데이터 필터링
  cond_more = (df_uncat_test['count'] >= n)                     # 빈도수 n이상 조건문
  list_more = df_uncat_test.loc[cond_more, '카드메시지'].to_list() # 조건문에 해당하는 데이터 리스트형태로 추출
  sum_count_more = df_uncat_test.loc[cond_more, 'count'].sum()  # 각 카드메시지의 총 빈도수
  sum_amount_more = df_uncat_test.loc[cond_more, 'sum'].sum()   # 각 카드메시지의 총 결제금액

  # 비율확인
  print("------ 빈도수가 {0}이상인 카드메시지 정보 ------".format(n))
  print("종류 갯수 {0}".format( len(list_more) ) )
  print("종류 비율 {0}%".format( 100 * round(len(list_more)/df['카드메시지'].nunique(), 4)) )
  print("데이터 비율 {0}%".format( 100 * round( sum_count_more / len(df), 4)) )
  print("결제액 비율 {0}%".format( 100 * round( sum_amount_more / df['결제금액2'].sum(), 4)) )

  # 인덱스 추출 및 데이터에서 제거
  ind_onlyone = [ind for ind, val in enumerate(df['카드메시지']) if val not in list_more] # 빈도수 n이상인 인덱스 추출
  df.drop(ind_onlyone, axis = 0, inplace = True)                                       # 해당 데이터 제거 (Row)
  df.reset_index(drop = True, inplace = True)                                          # 인덱스 리셋 (인덱스 컬럼 제거)
  return df
df_uncat_100 = df_freq_basis(df_uncat, n = 4)

------ 빈도수가 4이상인 카드메시지 정보 ------
종류 갯수 42286
종류 비율 61.519999999999996%
데이터 비율 87.75%
결제액 비율 52.61%


In [8]:
import pandas as pd
import numpy as np

df_uncat_more4 = pd.read_csv('미분류_빈도수4이상.csv')
df_uncat_more4.head(10)

,결제금액2,카드메시지,브랜드명,말뭉치3,토큰화
0,200000,경주페이,미분류,"경주페이,경주페이,경주페이,","['경주페이', '경주페이', '경주페이']"
1,22100,신한카드,미분류,"신한카드,신한카드,신한카드,","['신한카드', '신한카드', '신한카드']"
2,1500,분당서울대학,미분류,"분당서울대학,분당서울대학,분당서울대학,","['분당서울대학', '분당서울대학', '분당서울대학']"
3,200000,전지현,미분류,"전지현,전지현,전지현,","['전지현', '전지현', '전지현']"
4,25000,김설아,미분류,"김설아,김설아,김설아,","['김설아', '김설아', '김설아']"
5,24800,KB카드출금,미분류,"KB카드출금,KB카드출금,KB카드출금,","['KB카드출금', 'KB카드출금', 'KB카드출금']"
6,22100,금성커피,미분류,"금성커피,금성커피,금성커피,","['금성커피', '금성커피', '금성커피']"
7,39590,지에스칼텍,미분류,"지에스칼텍,지에스칼텍,지에스칼텍,","['지에스칼텍', '지에스칼텍', '지에스칼텍']"
8,43587,신도시충전소,미분류,"신도시충전소,신도시충전소,신도시충전소,","['신도시충전소', '신도시충전소', '신도시충전소']"
9,900,06월입출통,미분류,"06월입출통,06월입출통,06월입출통,","['06월입출통', '06월입출통', '06월입출통']"


In [9]:
# 단어 이어 붙여서 말뭉치 만들기
def get_corpus(df, num_duplicate):
  df['말뭉치3'] = df['카드메시지'].apply(lambda x: num_duplicate * (x + ','))
  df['토큰화'] = df['말뭉치3'].apply(lambda x: x.split(',')) # comma로 나누기
  df['토큰화'] = df['토큰화'].apply(lambda x: x[:-1]) # 마지막 comma 지우기
  return df
df_uncat_more4 = get_corpus(df_uncat_more4,  num_duplicate = 30)
df_uncat_more4.head()

,결제금액2,카드메시지,브랜드명,말뭉치3,토큰화
0,200000,경주페이,미분류,"경주페이,경주페이,경주페이,경주페이,경주페이,경주페이,경주페이,경주페이,경주페이,경...","[경주페이, 경주페이, 경주페이, 경주페이, 경주페이, 경주페이, 경주페이, 경주페..."
1,22100,신한카드,미분류,"신한카드,신한카드,신한카드,신한카드,신한카드,신한카드,신한카드,신한카드,신한카드,신...","[신한카드, 신한카드, 신한카드, 신한카드, 신한카드, 신한카드, 신한카드, 신한카..."
2,1500,분당서울대학,미분류,"분당서울대학,분당서울대학,분당서울대학,분당서울대학,분당서울대학,분당서울대학,분당서울...","[분당서울대학, 분당서울대학, 분당서울대학, 분당서울대학, 분당서울대학, 분당서울대..."
3,200000,전지현,미분류,"전지현,전지현,전지현,전지현,전지현,전지현,전지현,전지현,전지현,전지현,전지현,전지...","[전지현, 전지현, 전지현, 전지현, 전지현, 전지현, 전지현, 전지현, 전지현, ..."
4,25000,김설아,미분류,"김설아,김설아,김설아,김설아,김설아,김설아,김설아,김설아,김설아,김설아,김설아,김설...","[김설아, 김설아, 김설아, 김설아, 김설아, 김설아, 김설아, 김설아, 김설아, ..."


# 미분류 데이터

In [ ]:
import gensim
from gensim.models.fasttext import FastText

# stemmed = ['data', 'appli', 'scientist', ... ]

# def gen_words(stemmed):
#     yield stemmed   

model = FastText(df_uncat_more4['토큰화'],  window=3, min_count= 1, workers=3, sg=1)

model.train(df_uncat_more4['토큰화'], total_examples=model.corpus_count, epochs=3)

(44288784, 51275430)

In [10]:
import gensim
from gensim.models.fasttext import FastText

# model.save('new_more4_ft30.model')

embedding_mod = FastText.load('new_more4_ft30.model')
print(embedding_mod.wv['GS25'])
print(embedding_mod.wv.most_similar('GS25'))

[-0.230248    0.7292738  -0.04774703  0.30942732  0.15261415  0.29103148
  0.00487389  0.17448452 -0.5590059  -0.15555947  1.3700291  -0.10761707
  0.1364467  -0.45913634  0.48568887  0.29364657 -0.17286837  0.15223181
  0.15188682 -0.23741646  0.48055068  0.39232752  0.39268845  0.02057677
  0.0455237   0.5342798   0.42907953 -0.802318    0.5981428   0.4322912
 -0.44241905  0.67457443 -0.149635    0.33068255  0.71649915  0.19145623
 -0.06126576 -0.0553552   0.5798854  -0.7653518   0.37201175 -0.30609232
  0.26498964 -0.44119295  0.25444728 -0.2079237  -0.2619398  -0.44014415
 -0.06163533  0.4461403   0.01778777  0.06790302 -0.38225287  0.5715226
  0.13144386  0.1067758  -0.6436845  -0.05063017 -0.9523881   0.11542107
  0.09030814 -0.36603993  0.10697041 -0.07684562 -0.38336843  0.4665684
  0.32284373 -0.64515764  0.7061063   0.41184062  0.20137624 -0.3139488
 -0.14497854  0.27378333 -0.20748802 -0.2972624   0.20344777 -0.67801017
 -0.04147855  0.0869987  -0.31297383 -0.24559936  0.755

In [6]:
embedding_mod.wv.most_similar('신한', topn=30)

[('신한교보', 0.9934428930282593),
 ('신한정종옥노', 0.9922605156898499),
 ('신한목재', 0.9910688996315002),
 ('신한노종욱', 0.9886409044265747),
 ('신한나', 0.9878530502319336),
 ('신한청약', 0.9874107241630554),
 ('신한캠코김주', 0.9854938387870789),
 ('신한이자', 0.9847435355186462),
 ('신한주피알앤', 0.9846897125244141),
 ('신한에이피엘로', 0.98463374376297),
 ('신한포장주', 0.9836443066596985),
 ('신한철', 0.9834347367286682),
 ('신한은행위현철', 0.9827243685722351),
 ('신한오픈김상', 0.9826512932777405),
 ('신한은행김현성', 0.9825711846351624),
 ('신한이민선', 0.9825092554092407),
 ('신한김태훈', 0.9817471504211426),
 ('신한서울전자산', 0.9815651178359985),
 ('신한할부금융', 0.9809380769729614),
 ('신한흥국생명보', 0.9805570244789124),
 ('신한오픈윤영순', 0.9801684021949768),
 ('신한은행', 0.9798412322998047),
 ('신한리드코프', 0.9795363545417786),
 ('신한오픈허강일', 0.9788482785224915),
 ('신한금융투자', 0.9780742526054382),
 ('신한BC', 0.9777829647064209),
 ('신한오픈라인', 0.9775990843772888),
 ('신한생보험', 0.977263867855072),
 ('신한주넥스젠', 0.9765298962593079),
 ('신한오픈안수향', 0.9763913750648499)]

## Unique 카드메시지 groupby()

In [11]:
def df_unique(df, num_duplicate):
  list_unique_message = list(df['카드메시지'].unique()) 
  df_uncat_unique = pd.DataFrame()
  df_uncat_unique['카드메시지'] = list_unique_message
  

  # 단어 이어 붙여서 말뭉치 만들기
  df_uncat_unique['말뭉치3'] = df_uncat_unique['카드메시지'].apply(lambda x: num_duplicate * (str(x) + ','))
  df_uncat_unique['토큰화'] = df_uncat_unique['말뭉치3'].apply(lambda x: x.split(',')) # comma로 나누기
  df_uncat_unique['토큰화'] = df_uncat_unique['토큰화'].apply(lambda x: x[:-1]) # 마지막 comma 지우기
  return df_uncat_unique

df_uncat_unique = df_unique(df_uncat_more4, num_duplicate = 30)
print('데이터 수: ', len(df_uncat_unique))
df_uncat_unique

데이터 수:  42286


,카드메시지,말뭉치3,토큰화
0,경주페이,"경주페이,경주페이,경주페이,경주페이,경주페이,경주페이,경주페이,경주페이,경주페이,경...","[경주페이, 경주페이, 경주페이, 경주페이, 경주페이, 경주페이, 경주페이, 경주페..."
1,신한카드,"신한카드,신한카드,신한카드,신한카드,신한카드,신한카드,신한카드,신한카드,신한카드,신...","[신한카드, 신한카드, 신한카드, 신한카드, 신한카드, 신한카드, 신한카드, 신한카..."
2,분당서울대학,"분당서울대학,분당서울대학,분당서울대학,분당서울대학,분당서울대학,분당서울대학,분당서울...","[분당서울대학, 분당서울대학, 분당서울대학, 분당서울대학, 분당서울대학, 분당서울대..."
3,전지현,"전지현,전지현,전지현,전지현,전지현,전지현,전지현,전지현,전지현,전지현,전지현,전지...","[전지현, 전지현, 전지현, 전지현, 전지현, 전지현, 전지현, 전지현, 전지현, ..."
4,김설아,"김설아,김설아,김설아,김설아,김설아,김설아,김설아,김설아,김설아,김설아,김설아,김설...","[김설아, 김설아, 김설아, 김설아, 김설아, 김설아, 김설아, 김설아, 김설아, ..."
...,...,...,...
42281,이화의대부속목동총,"이화의대부속목동총,이화의대부속목동총,이화의대부속목동총,이화의대부속목동총,이화의대부속...","[이화의대부속목동총, 이화의대부속목동총, 이화의대부속목동총, 이화의대부속목동총, 이..."
42282,김명자굴국,"김명자굴국,김명자굴국,김명자굴국,김명자굴국,김명자굴국,김명자굴국,김명자굴국,김명자굴...","[김명자굴국, 김명자굴국, 김명자굴국, 김명자굴국, 김명자굴국, 김명자굴국, 김명자..."
42283,주식회사잉크,"주식회사잉크,주식회사잉크,주식회사잉크,주식회사잉크,주식회사잉크,주식회사잉크,주식회사...","[주식회사잉크, 주식회사잉크, 주식회사잉크, 주식회사잉크, 주식회사잉크, 주식회사잉..."
42284,라밥LABAB대방점,"라밥LABAB대방점,라밥LABAB대방점,라밥LABAB대방점,라밥LABAB대방점,라밥...","[라밥LABAB대방점, 라밥LABAB대방점, 라밥LABAB대방점, 라밥LABAB대방..."


In [12]:
# 길이 3인 카드메시지 제거 (이름이라 판단)
# 인덱스 추출 및 데이터에서 제거
ind_3length = [ind for ind, val in enumerate(df_uncat_unique['카드메시지']) if len(val) == 3]
df_uncat_unique.drop(ind_3length, axis = 0, inplace = True)
df_uncat_unique.reset_index(drop = True, inplace = True)

In [13]:
df_uncat_unique

,카드메시지,말뭉치3,토큰화
0,경주페이,"경주페이,경주페이,경주페이,경주페이,경주페이,경주페이,경주페이,경주페이,경주페이,경...","[경주페이, 경주페이, 경주페이, 경주페이, 경주페이, 경주페이, 경주페이, 경주페..."
1,신한카드,"신한카드,신한카드,신한카드,신한카드,신한카드,신한카드,신한카드,신한카드,신한카드,신...","[신한카드, 신한카드, 신한카드, 신한카드, 신한카드, 신한카드, 신한카드, 신한카..."
2,분당서울대학,"분당서울대학,분당서울대학,분당서울대학,분당서울대학,분당서울대학,분당서울대학,분당서울...","[분당서울대학, 분당서울대학, 분당서울대학, 분당서울대학, 분당서울대학, 분당서울대..."
3,KB카드출금,"KB카드출금,KB카드출금,KB카드출금,KB카드출금,KB카드출금,KB카드출금,KB카드...","[KB카드출금, KB카드출금, KB카드출금, KB카드출금, KB카드출금, KB카드출..."
4,금성커피,"금성커피,금성커피,금성커피,금성커피,금성커피,금성커피,금성커피,금성커피,금성커피,금...","[금성커피, 금성커피, 금성커피, 금성커피, 금성커피, 금성커피, 금성커피, 금성커..."
...,...,...,...
29140,스마트하우스,"스마트하우스,스마트하우스,스마트하우스,스마트하우스,스마트하우스,스마트하우스,스마트하...","[스마트하우스, 스마트하우스, 스마트하우스, 스마트하우스, 스마트하우스, 스마트하우..."
29141,이화의대부속목동총,"이화의대부속목동총,이화의대부속목동총,이화의대부속목동총,이화의대부속목동총,이화의대부속...","[이화의대부속목동총, 이화의대부속목동총, 이화의대부속목동총, 이화의대부속목동총, 이..."
29142,김명자굴국,"김명자굴국,김명자굴국,김명자굴국,김명자굴국,김명자굴국,김명자굴국,김명자굴국,김명자굴...","[김명자굴국, 김명자굴국, 김명자굴국, 김명자굴국, 김명자굴국, 김명자굴국, 김명자..."
29143,주식회사잉크,"주식회사잉크,주식회사잉크,주식회사잉크,주식회사잉크,주식회사잉크,주식회사잉크,주식회사...","[주식회사잉크, 주식회사잉크, 주식회사잉크, 주식회사잉크, 주식회사잉크, 주식회사잉..."


In [14]:
# df_uncat_100['fasttest_wv'] = df_uncat_100['카드메시지'].apply(lambda x: embedding_mod.wv[x].tolist())
df_uncat_unique['fasttest_wv'] = df_uncat_unique['카드메시지'].apply(lambda x: embedding_mod.wv[x].tolist())


## DBSCAN (Density-based Spatial Clustering of Applications with Noise)

**Two Key Parameters**

- eps: 이웃을 구체화시키는 거리기준

- minPts: 집단을 이루는 최소 데이터의 양


**Classified Points**

- Core point: 중심으로 집단을 이루는 점

- Border point: core point와 닿지만 주변의 점이 minPts보다 적은 점

- Outlier: 어떠한 core point와도 닿지 않는 점

X = np.array([[1, 2], [2, 2], [2, 3],
...               [8, 7], [8, 8], [25, 80]])

In [17]:
# input data array 만들기
X = []
for i in df_uncat_unique['fasttest_wv']:
  X.append(i)
len(X)

29145

In [18]:
from sklearn.cluster import DBSCAN
min_sample = []
num_clustered = []
ratio_clustered = []
cluster_group = []
for i in range(2, 10,1):
  print('--------------------')
  print(f"최소 샘플 갯수: {i}")
  dbscan = DBSCAN(eps = 0.01, min_samples=i, metric='cosine')
  clustered = dbscan.fit_predict(X)
  print('클러스터링 그룹 갯수: ', len(np.unique(clustered)))
  num_unique_card = len(df_uncat_unique['카드메시지'].unique())
  print('카드메시지 종류 갯수: ', num_unique_card)
  print('분류비율: {0}%'.format( 100 * round(len(np.unique(clustered)) / num_unique_card, 4) ) )

  # 결과 데이터프레임에 저장
  result_df = pd.DataFrame()
  ## 샘플 수
  min_sample.append(i)
  ## 클러스터링 그룹 갯수
  num_clustered.append( len(np.unique(clustered)) )
  ## 분류비율
  ratio_clustered.append(100 * round(len(np.unique(clustered)) / num_unique_card, 4))
  ## 클러스터링 결과 array
  cluster_group.append(clustered)

## 결과 저장
result_df['샘플수'] = min_sample
result_df['클러스터링_그룹갯수'] = num_clustered
result_df['분류비율'] = ratio_clustered
result_df['결과'] = cluster_group

--------------------
최소 샘플 갯수: 2
클러스터링 그룹 갯수:  1730
카드메시지 종류 갯수:  29145
분류비율: 5.94%
--------------------
최소 샘플 갯수: 3
클러스터링 그룹 갯수:  844
카드메시지 종류 갯수:  29145
분류비율: 2.9000000000000004%
--------------------
최소 샘플 갯수: 4
클러스터링 그룹 갯수:  557
카드메시지 종류 갯수:  29145
분류비율: 1.91%
--------------------
최소 샘플 갯수: 5
클러스터링 그룹 갯수:  407
카드메시지 종류 갯수:  29145
분류비율: 1.4000000000000001%
--------------------
최소 샘플 갯수: 6
클러스터링 그룹 갯수:  338
카드메시지 종류 갯수:  29145
분류비율: 1.16%
--------------------
최소 샘플 갯수: 7
클러스터링 그룹 갯수:  275
카드메시지 종류 갯수:  29145
분류비율: 0.9400000000000001%
--------------------
최소 샘플 갯수: 8
클러스터링 그룹 갯수:  245
카드메시지 종류 갯수:  29145
분류비율: 0.84%
--------------------
최소 샘플 갯수: 9
클러스터링 그룹 갯수:  213
카드메시지 종류 갯수:  29145
분류비율: 0.73%


In [19]:
def get_cluster(df_uncat_more4, df_uncat_unique, n):
  # -----------------
  # df_uncat_more4 = 빈도수 4 이상인 데이터
  # df_uncat_unique = unique한 카드메시지만 담은 데이터프레임
  # n = n번째 클러스터링 결과 (from result_df)
  # -----------------
  clusted_data = df_uncat_unique.copy()
  clusted_data['clustered'] = result_df.loc[n ,'결과'].tolist()
  clustered_result = clusted_data.loc[:, ['카드메시지', 'clustered']]

  # 빈도수 4이상 데이터에서 카드메시지와 결제금액2 만 복사
  df_result = df_uncat_more4.loc[:, ['카드메시지', '결제금액2']]
  # 빈도수 4이상데이터와 클러스터링 결과 결합
  final_result = df_result.merge(clustered_result, how = 'left', on='카드메시지')
  # 분류 안된 데이터 '-1'로 대체
  final_result['clustered'].fillna(-1, inplace = True)
  return clustered_result, final_result
clustered_result, final_result = get_cluster(df_uncat_more4, df_uncat_unique, n = 0)

In [20]:
clustered_result.head()

,카드메시지,clustered
0,경주페이,-1
1,신한카드,-1
2,분당서울대학,-1
3,KB카드출금,-1
4,금성커피,0


In [21]:
final_result.head()

,카드메시지,결제금액2,clustered
0,경주페이,200000,-1.0
1,신한카드,22100,-1.0
2,분당서울대학,1500,-1.0
3,전지현,200000,-1.0
4,김설아,25000,-1.0


In [22]:
categorized = (final_result['clustered'] != -1)
cat_result_df = final_result[categorized]
cat_result_df

,카드메시지,결제금액2,clustered
6,금성커피,22100,0.0
15,UTSKPay5,6100,1.0
17,포털건강보험공,302400,2.0
18,아인헤어,20000,3.0
22,주식회사홈마트,5800,4.0
...,...,...,...
569706,아르코펫코리아,121000,389.0
569707,주식회사우,25400,236.0
569720,영계소문,10500,1413.0
569724,하나카드금융,173215,782.0


In [23]:
ratio_data = 100 * round( len(cat_result_df) / len(final_result) , 4)
ratio_sum = 100 * round( cat_result_df['결제금액2'].sum() / final_result['결제금액2'].sum() , 4)

print(f'전체 분류된 데이터 비율: { ratio_data}%')
print(f'전체 분류된 데이터 금액 비율: { ratio_sum }%')

전체 분류된 데이터 비율: 20.22%
전체 분류된 데이터 금액 비율: 6.09%


In [24]:
num_unique_cluster = len(cat_result_df['clustered'].unique()) # 분류된 클러스터링 그룹 갯수
grouping_message = []
grouping_payment = []
for i in range(num_unique_cluster):
  cond = (cat_result_df['clustered'] == i) # i로 클러스터링 된 데이터를 추출하기 위한 조건문
  list_cat_message = cat_result_df.loc[cond, '카드메시지'].unique().tolist() # 같은 클러스터링에 존재하는 모든 카드메시지 유형 리스트형태로 변환
  grouping_message.append(list_cat_message)

  cat_payment = cat_result_df.loc[cond, '결제금액2'].sum()
  grouping_payment.append(cat_payment)


In [31]:
df_cat_message = pd.DataFrame()
df_cat_message['카드메시지들'] = grouping_message
df_cat_message['clustered'] = [i for i in range(num_unique_cluster)]
df_cat_message['결제합'] = grouping_payment
df_cat_message['결제%_분류만'] = 100 *round( df_cat_message['결제합'] / df_cat_message['결제합'].sum(), 6)
df_cat_message['결제%_전체'] = 100 * round(df_cat_message['결제합'] / final_result['결제금액2'].sum(), 6)
Result = df_cat_message.sort_values(by = '결제합', ascending=False)
Result

,카드메시지들,clustered,결제합,결제%_분류만,결제%_전체
20,"[주식회사마, 주식회사새날, 주식회사현, 주식회사상익유, 주식회사위총, 주식회사띵굴...",20,6291795572,17.4121,1.0604
58,"[상호저축은, 예가람저축은]",58,5111228481,14.1449,0.8614
1568,"[내부지급5512, 내부지급5504, 내부지급5515]",1568,3703488471,10.2491,0.6242
459,"[인터넷환전거래, 인터넷통신료, 인터넷지로]",459,2204082539,6.0996,0.3715
24,"[하나전영환은, 하나황재희통, 하나홍숙, 하나전인경풀, 하나김기량, 하나이재용, 하...",24,1262986347,3.4952,0.2129
...,...,...,...,...,...
1608,"[우신고등학교매점, 라우디스성남외국어고등학교매점]",1608,17700,0.0000,0.0000
1654,"[주비트코퍼레이션, 주비트코퍼레이]",1654,15800,0.0000,0.0000
910,"[수원시체육회, 수원시체육회매]",910,15100,0.0000,0.0000
847,"[화성시지역, 화성시지역개발사]",847,10750,0.0000,0.0000


In [32]:
Result.to_csv('최종결과물_아이디어웨어.csv', index = False)

## 요약

### FastText
- 3 이상의 빈도수를 가진 데이터로 FastText 훈련
- FastText 모델 저장 후 임베딩 벡터사용

### DBSCAN
- 처음 데이터에서 빈도수 20개 이상인 메시지만 필터링
- FastText 임베딩벡터 데이터프레임에 저장 (리스트형태로)
- DBSCAN 모델에 넣을 데이터 생성 (임베딩벡터를 하나의 리스트 형태로 만든 후 array로 변환)

    예) X = np.array([[1, 2], [2, 2], [2, 3],
    ...               [8, 7], [8, 8], [25, 80]])

- DBSCAN 모델 훈련 및 결과물 저장 (fit_predict 메소드를 사용)
  - 단, 데이터를 많이 넣으면 세션이 멈춰서 10만개만 넣음

- 결과를 데이터 프레임에 붙여넣기 

### 결론
- FastText는 Unique한 카드메시지로 놓고 돌리기 보다는 빈도수를 기준으로 데이터를 전처리해서 돌리는 것이 좋다.
- FastText는 빈도수가 3개이상인 데이터만 놓고 돌렸을 경우가 가장 좋다
- 데이터 약 26만개 중 10만개 기준으로 분류한 결과
- 2216개의 카드메시지 종류에서 1643개의 그룹생성 (25% 감소효과)

### 한계점

- FastText임베딩값에 따라 그룹화가 달라진다
- 단어형태를 억지로 말뭉치형태로 바꾸어 진행하였기에 성능문제가 발생할 수 있다
- 분류를 했지만 사람이 분류된 그룹을 직접 확인하여 세부작업을 진행해야 한다.
- 성능지표가 따로 없어서 모델의 성능을 볼 수 없다.